<a href="https://colab.research.google.com/github/Samuel-CHLam/Solving_PDE_by_shallow_NN/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd ./drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
%cd ./kolmogorov-master-v2

/content/drive/MyDrive/kolmogorov-master-v2


In [ ]:
import numpy as np
import tensorflow as tf
import time
from tqdm import tqdm
from tensorflow.python.ops import init_ops
from tensorflow.compat.v1.keras import initializers
from tensorflow.python.training.moving_averages import assign_moving_average
tf.compat.v1.disable_eager_execution()

In [ ]:
def neural_net(x, neurons, is_training, name,
               mv_decay=0.9, dtype=tf.float32):

    def _batch_normyalization(_x):
        beta = tf.compat.v1.get_variable('beta', [_x.get_shape()[-1]],
                               dtype, init_ops.zeros_initializer())
        gamma = tf.compat.v1.get_variable('gamma', [_x.get_shape()[-1]],
                                dtype, init_ops.ones_initializer())
        mv_mean = tf.compat.v1.get_variable('mv_mean', [_x.get_shape()[-1]],
                                  dtype, init_ops.zeros_initializer(),
                                  trainable=False)
        mv_variance = tf.compat.v1.get_variable('mv_variance', [_x.get_shape()[-1]],
                                      dtype, init_ops.ones_initializer(),
                                      trainable=False)
        mean, variance = tf.nn.moments(x=_x, axes=[0], name='moments')
        tf.compat.v1.add_to_collection(tf.compat.v1.GraphKeys.UPDATE_OPS,
                             assign_moving_average(mv_mean, mean,
                                                   mv_decay, True))
        tf.compat.v1.add_to_collection(tf.compat.v1.GraphKeys.UPDATE_OPS,
                             assign_moving_average(mv_variance, variance,
                                                   mv_decay, False))
        mean, variance = tf.cond(pred=is_training,
                                 true_fn=lambda: (mean, variance),
                                 false_fn=lambda: (mv_mean, mv_variance))
        return tf.nn.batch_normalization(_x, mean, variance,
                                         beta, gamma, 1e-6)

    def _layer(_x, out_size, activation_fn):
        w = tf.compat.v1.get_variable('weights',
                            [_x.get_shape().as_list()[-1], out_size],
                            dtype, initializers.glorot_normal())
        return activation_fn(_batch_normalization(tf.matmul(_x, w)))

    with tf.compat.v1.variable_scope(name):
        x = _batch_normalization(x)
        for i in range(len(neurons)):
            with tf.compat.v1.variable_scope(f'layer_{i + 1}_'):
                x = _layer(x, neurons[i],
                           tf.nn.tanh if i < len(neurons)-1 else tf.identity)
    return x

In [ ]:
def kolmogorov_train_and_test(xi, x_sde, phi, u_reference, neurons,
                              lr_boundaries, lr_values, train_steps,
                              mc_rounds, mc_freq, file_name,
                              dtype=tf.float32):

    def _approximate_errors():
        lr, gs = sess.run([learning_rate, global_step])
        l1_err, l2_err, li_err = 0., 0., 0.
        rel_l1_err, rel_l2_err, rel_li_err = 0., 0., 0.
        for _ in range(mc_rounds):
            l1, l2, li, rl1, rl2, rli \
                = sess.run([err_l_1, err_l_2, err_l_inf,
                            rel_err_l_1, rel_err_l_2, rel_err_l_inf],
                           feed_dict={is_training: False})
            l1_err, l2_err, li_err = (l1_err + l1, l2_err + l2,
                                      np.maximum(li_err, li))
            rel_l1_err, rel_l2_err, rel_li_err \
                = (rel_l1_err + rl1, rel_l2_err + rl2,
                   np.maximum(rel_li_err, rli))
        l1_err, l2_err = l1_err / mc_rounds, np.sqrt(l2_err / mc_rounds)
        rel_l1_err, rel_l2_err \
            = rel_l1_err / mc_rounds, np.sqrt(rel_l2_err / mc_rounds)
        t_mc = time.time()
        file_out.write('%i, %f, %f, %f, %f, %f, %f, %f, '
                       '%f, %f\n' % (gs, l1_err,  l2_err, li_err,
                                     rel_l1_err, rel_l2_err, rel_li_err, lr,
                                     t1_train - t0_train, t_mc - t1_train))
        file_out.flush()

    t0_train = time.time()
    is_training = tf.compat.v1.placeholder(tf.bool, [])
    u_approx = neural_net(xi, neurons, is_training, 'u_approx', dtype=dtype)
    loss = tf.reduce_mean(input_tensor=tf.math.squared_difference(u_approx, phi(x_sde)))

    err = tf.abs(u_approx - u_reference)
    err_l_1 = tf.reduce_mean(input_tensor=err)
    err_l_2 = tf.reduce_mean(input_tensor=err ** 2)
    err_l_inf = tf.reduce_max(input_tensor=err)
    rel_err = err / tf.maximum(u_reference, 1e-8)
    rel_err_l_1 = tf.reduce_mean(input_tensor=rel_err)
    rel_err_l_2 = tf.reduce_mean(input_tensor=rel_err ** 2)
    rel_err_l_inf = tf.reduce_max(input_tensor=rel_err)

    global_step = tf.compat.v1.get_variable('global_step', [], tf.int32,
                                  tf.compat.v1.constant_initializer(0),
                                  trainable=False)
    learning_rate = tf.compat.v1.train.piecewise_constant(global_step,
                                                lr_boundaries,
                                                lr_values)
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate)
    update_ops = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.UPDATE_OPS, 'u_approx')
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss, global_step)

    file_out = open(file_name, 'w')
    file_out.write('step, l1_err, l2_err, li_err, l1_rel, '
                   'l2_rel, li_rel, learning_rate, time_train, time_mc\n')

    with tf.compat.v1.Session() as sess:

        sess.run(tf.compat.v1.global_variables_initializer())

        for step in tqdm(range(train_steps)):
            if step % mc_freq == 0:
                t1_train = time.time()
                _approximate_errors()
                t0_train = time.time()
            sess.run(train_op, feed_dict={is_training: True})
        t1_train = time.time()
        _approximate_errors()

    file_out.close()

# Example

## Heat Equation

In [ ]:
tf.compat.v1.reset_default_graph()
dtype = tf.float32
T, N, d = 1., 1, 100
batch_size = 8192
neurons = [20, 1]
train_steps = 300000
mc_rounds, mc_freq = 1250, 100
lr_boundaries = [250001, 500001]
lr_values = [0.001, 0.0001, 0.00001]
xi = tf.random.uniform(shape=(batch_size, d), minval=0.,
                       maxval=1., dtype=dtype)
x_sde = xi + tf.random.normal(shape=(batch_size, d),
                              stddev=np.sqrt(2. * T / N), dtype=dtype)

def phi(x):
    return tf.reduce_sum(input_tensor=x ** 2, axis=1, keepdims=True)

u_reference = phi(xi) + 2. * T * d

kolmogorov_train_and_test(xi, x_sde, phi, u_reference, neurons,
                          lr_boundaries, lr_values, train_steps,
                          mc_rounds, mc_freq, 'example_heat_equation_single_20.csv', dtype)

100%|██████████| 300000/300000 [1:57:14<00:00, 42.65it/s]


## Geometric Brownian Motion

In [ ]:
tf.compat.v1.reset_default_graph()
dtype = tf.float32
T, N, d = 1., 1, 100
r, c, K = 0.05, 0.1, 100.
sigma = tf.constant(0.1 + 0.5 * np.linspace(start=1. / d, stop=1., num=d, endpoint=True), dtype=dtype)
batch_size = 8192
neurons = [d + 100, d + 100, 1]
train_steps = 300000
# mc_rounds, mc_freq = 1250, 100
mc_rounds, mc_freq = 10, 25000
mc_samples_ref, mc_rounds_ref = 1024, 1024
lr_boundaries = [250001, 500001]
lr_values = [0.001, 0.0001, 0.00001]
xi = tf.random.uniform((batch_size, d), minval=90., maxval=110., dtype=dtype)


def phi(x, axis=1):
    return tf.exp(-r * T) \
           * tf.maximum(tf.reduce_max(input_tensor=x, axis=axis, keepdims=True) - K, 0.)


def mc_body(idx, p):
    _x = xi * tf.exp((r - c - 0.5 * sigma ** 2) * T + sigma
                     * tf.random.normal((mc_samples_ref, batch_size, d),
                                        stddev=tf.sqrt(T / N), dtype=dtype))
    return idx + 1, p + tf.reduce_mean(input_tensor=phi(_x, 2), axis=0)


x_sde = xi * tf.exp((r - c - 0.5 * sigma ** 2) * T
                    + sigma * tf.random.normal((batch_size, d),
                                               stddev=tf.sqrt(T / N),
                                               dtype=dtype))
_, u = tf.while_loop(cond=lambda idx, p: idx < mc_rounds_ref, body=mc_body,
                     loop_vars=(tf.constant(0), tf.zeros((batch_size, 1), dtype)))
u_reference = u / tf.cast(mc_rounds_ref, tf.float32)

kolmogorov_train_and_test(xi, x_sde, phi, u_reference, neurons,
                          lr_boundaries, lr_values, train_steps,
                          mc_rounds, mc_freq, 'example_geometric_brownian_motions.csv', dtype)

  8%|▊         | 25000/300000 [36:49<6:45:08, 11.31it/s]


KeyboardInterrupt: ignored